## Kaggel: Digit Recognizer(MNIST)  by Talos  
Kaggle Digit recognizer: https://www.kaggle.com/c/digit-recognizer  
Talos: https://github.com/autonomio/talos  

### score:

In [1]:
import talos as ta

from keras.models import Sequential
from keras.layers import Dense, Dropout, Reshape, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import Adam
from keras.utils import np_utils
import keras

import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

# fix random seed
import tensorflow as tf
import random as rn
import os
os.environ['PYTHONHASHSEED'] = '0'
seed = 123
rn.seed(seed)
np.random.seed(seed)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
from keras import backend as K
tf.set_random_seed(seed)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


## Data preparation:
* Split data into train and test at 0.2
* dtype: float32
* Normalization
* One-hot labels

In [2]:
x_train = pd.read_csv('../train.csv')
y_train = x_train.label
x_train = x_train.drop(['label'], axis=1)

# from keras.datasets import mnist
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

# x_train, x_val, y_train, y_val = train_test_split(train, label, test_size=0.2, random_state=seed)

x_train = x_train.astype('float32')
x_train /= 255.0
#x_train = x_train.reshape(-1,784,)

x_train = x_train.values
y_train = y_train.values
#nb_classes = 10 
#y_train = np_utils.to_categorical(y_train, nb_classes)

In [3]:
[x_train.shape, y_train.shape]

[(10000, 784), (10000,)]

In [4]:
# y_train = y_train.values
type(y_train)

numpy.ndarray

## Model preparation:

In [5]:
from talos.model.early_stopper import early_stopper
#from talos.model.normalizers import lr_normalizer

### CNN model:

In [6]:
def cnn_model(x_train, y_train, x_val, y_val, params):
    initializer = keras.initializers.glorot_uniform(seed=seed)
    
    model = Sequential() 
    model.add(Reshape((28,28,1), input_shape=(784,)))
    
    model.add(Conv2D(32, (5,5), padding='same', activation='relu', kernel_initializer=initializer))
    model.add(Conv2D(32, (5,5), padding='same', activation='relu', kernel_initializer=initializer))
    model.add(MaxPool2D(pool_size=(2,2)))
    if(params['batchnorm_0']):
        model.add(BatchNormalization())
    model.add(Dropout(params['dropout_0'], seed=seed))
    
    model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_initializer=initializer))
    model.add(Conv2D(64, (3,3), padding='same', activation='relu', kernel_initializer=initializer))
    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    if(params['batchnorm_1']):
        model.add(BatchNormalization())
    model.add(Dropout(params['dropout_1'], seed=seed))

    model.add(Flatten())
    model.add(Dense(params['dense_0'], activation="relu", kernel_initializer=initializer))
    if(params['batchnorm_2']):
        model.add(BatchNormalization())
    model.add(Dropout(params['dropout_2'], seed=seed))
    model.add(Dense(params['dense_1'], activation = "relu", kernel_initializer=initializer))
    if(params['batchnorm_3']) == 'bn31':
        model.add(BatchNormalization())
    model.add(Dropout(params['dropout_3'], seed=seed))
    model.add(Dense(10, activation = "softmax", kernel_initializer=initializer))

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['acc'])

    hist = model.fit(x_train, y_train,
                     batch_size=params['batch_size'],
                     epochs=params['epochs'],
                     verbose=1,
                     validation_data=[x_val, y_val],
                     callbacks=early_stopper(params['epochs'], mode='strict'))
    
    return hist, model

## Parameter space boundaries:

In [7]:
p = {'batchnorm_0':[0, 1],
     'batchnorm_1':[0, 1],
     'batchnorm_2':[0, 1],
     'batchnorm_3':[0, 1],
     'dropout_0':[0, 0.25, 0.5],
     'dropout_1':[0, 0.25, 0.5],
     'dropout_2':[0, 0.25, 0.5],
     'dropout_3':[0, 0.25, 0.5],
     'dense_0': [64, 128, 256],
     'dense_1': [16, 32, 64],
     'batch_size':[8, 16, 32],
     'epochs': [50]}

## Hyperparameter scan:

In [8]:
h = ta.Scan(x_train, y_train,
            params=p,
            model=cnn_model,
            dataset_name='mnist',
            experiment_no='1',
            grid_downsample=.01,
            seed=seed,
            val_split=0.2)

  0%|          | 0/432000 [00:00<?, ?it/s]

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 5s 657us/step - loss: 0.5873 - acc: 0.8074 - val_loss: 0.4096 - val_acc: 0.8900
Epoch 2/100
8000/8000 [==============================] - 4s 459us/step - loss: 0.2644 - acc: 0.9176 - val_loss: 0.1624 - val_acc: 0.9505
Epoch 3/100
8000/8000 [==============================] - 4s 458us/step - loss: 0.2238 - acc: 0.9331 - val_loss: 0.1147 - val_acc: 0.9665
Epoch 4/100
8000/8000 [==============================] - 4s 458us/step - loss: 0.1857 - acc: 0.9447 - val_loss: 0.1101 - val_acc: 0.9690
Epoch 5/100
8000/8000 [==============================] - 4s 458us/step - loss: 0.1818 - acc: 0.9465 - val_loss: 0.1231 - val_acc: 0.9645
Epoch 6/100
8000/8000 [==============================] - 4s 458us/step - loss: 0.1518 - acc: 0.9540 - val_loss: 0.0979 - val_acc: 0.9725
Epoch 7/100
8000/8000 [==============================] - 4s 459us/step - loss: 0.1316 - acc: 0.9609 - val_loss: 0.1079 - val_acc: 

  0%|          | 1/432000 [00:54<6586:36:48, 54.89s/it]

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 6s 728us/step - loss: 2.2151 - acc: 0.1603 - val_loss: 2.0373 - val_acc: 0.1895
Epoch 2/100
8000/8000 [==============================] - 5s 670us/step - loss: 2.1711 - acc: 0.1720 - val_loss: 2.3082 - val_acc: 0.1150
Epoch 3/100
8000/8000 [==============================] - 5s 661us/step - loss: 2.3027 - acc: 0.1016 - val_loss: 2.3029 - val_acc: 0.0990


  0%|          | 2/432000 [01:12<5239:25:21, 43.66s/it]

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 5s 617us/step - loss: 2.3044 - acc: 0.1076 - val_loss: 2.3065 - val_acc: 0.0990
Epoch 2/100
8000/8000 [==============================] - 5s 575us/step - loss: 2.3028 - acc: 0.1074 - val_loss: 2.3047 - val_acc: 0.0990
Epoch 3/100
8000/8000 [==============================] - 5s 566us/step - loss: 2.3031 - acc: 0.1105 - val_loss: 2.3049 - val_acc: 0.0990
Epoch 4/100
8000/8000 [==============================] - 5s 568us/step - loss: 2.3030 - acc: 0.1035 - val_loss: 2.3041 - val_acc: 0.0990
Epoch 5/100
8000/8000 [==============================] - 5s 567us/step - loss: 2.3026 - acc: 0.1064 - val_loss: 2.3071 - val_acc: 0.0990
Epoch 6/100
8000/8000 [==============================] - 5s 566us/step - loss: 2.3033 - acc: 0.1096 - val_loss: 2.3043 - val_acc: 0.0990


  0%|          | 3/432000 [01:40<4694:49:34, 39.12s/it]

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 7s 863us/step - loss: 1.6156 - acc: 0.4735 - val_loss: 0.5633 - val_acc: 0.8335
Epoch 2/100
1568/8000 [====>.........................] - ETA: 4s - loss: 1.0689 - acc: 0.6601

KeyboardInterrupt: 

In [ ]:
# h.data.head()

# h.peak_epochs_df

# h.details

In [ ]:
# h.saved_models

# h.saved_weights

## Save the Best model and Weights

In [ ]:
# best_model.save('talos_best_model.h5')
# best_model.save_weights('talos_best_model_weights.h5')

## Submission File

In [ ]:
# test = pd.read_csv('../test.csv')
# test_index = test.index

# from keras.models import load_model
# best_model = load_model('talos_best_model.h5')
# best_model.load_weights('talos_best_model_weights.h5')

# pred = best_model.predict(test)
# result = pred.argmax(axis=1)

# submission = pd.DataFrame({'ImageId': test_index+1, 'Label': result})
# submission.to_csv('talos_submission.csv', index=False)